In [2]:
import os
import sys
from pathlib import Path

import einops
import plotly.express as px
import torch as t
from IPython.display import display
from ipywidgets import interact
from jaxtyping import Bool, Float, jaxtyped
from torch import Tensor
from typeguard import typechecked as typechecker

# Make sure exercises are in the path
chapter = r"chapter0_fundamentals"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part1_ray_tracing"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

import part1_ray_tracing.tests as tests
from part1_ray_tracing.utils import (
    render_lines_with_plotly,
    setup_widget_fig_ray,
    setup_widget_fig_triangle,
)
from plotly_utils import imshow

MAIN = __name__ == "__main__"

In [3]:
def make_rays_1d(num_pixels: int, y_limit: float) -> t.Tensor:
    '''
    num_pixels: The number of pixels in the y dimension. Since there is one ray per pixel, this is also the number of rays.
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both endpoints.

    Returns: shape (num_pixels, num_points=2, num_dim=3) where the num_points dimension contains (origin, direction) and the num_dim dimension contains xyz.

    Example of make_rays_1d(9, 1.0): [
        [[0, 0, 0], [1, -1.0, 0]],
        [[0, 0, 0], [1, -0.75, 0]],
        [[0, 0, 0], [1, -0.5, 0]],
        ...
        [[0, 0, 0], [1, 0.75, 0]],
        [[0, 0, 0], [1, 1, 0]],
    ]
    '''
    zeroes = t.zeros((num_pixels,2, 3))

    #y_limit = 1.0
    #num_pixels = 9.0
    #num_steps = (num_pixels - 1)
    #step_size = (2 * y_limit) / num_steps


    #t.arange(-y_limit, y_limit + step_size, (2 * y_limit) / num_steps, out=zeroes[:, 1, 1])
    t.linspace(-y_limit, y_limit, int(num_pixels),  out=zeroes[:, 1, 1])
    zeroes[:, 1, 0] = 1

    return zeroes
    

rays1d = make_rays_1d(9, 10.0)

print(rays1d)

fig = render_lines_with_plotly(rays1d)

tensor([[[  0.0000,   0.0000,   0.0000],
         [  1.0000, -10.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -7.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -5.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  -2.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   0.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   2.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   5.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,   7.5000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000],
         [  1.0000,  10.0000,   0.0000]]])


In [4]:
segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]], 
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]], 
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])

fig = render_lines_with_plotly(t.cat((rays1d, segments)))

In [27]:
def intersect_ray_1d(ray: t.Tensor, segment: t.Tensor) -> bool:
    """
    ray: shape (n_points=2, n_dim=3)  # O, D points
    segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

    Return True if the ray intersects the segment.
    """
    origin = ray[0][:2]
    direction = ray[1][:2]
    l1 = segment[0][:2]
    l2 = segment[1][:2]

    try:
        solution_vector = t.linalg.solve(t.stack((direction, l1 - l2), dim=1), l1 - origin)
    except:
        return False

    u = solution_vector[0]
    v = solution_vector[1]

    return u >= 0.0 and v >= 0.0 and v <= 1.0


tests.test_intersect_ray_1d(intersect_ray_1d)
tests.test_intersect_ray_1d_special_case(intersect_ray_1d)

All tests in `test_intersect_ray_1d` passed!
All tests in `test_intersect_ray_1d_special_case` passed!
